<a href="https://colab.research.google.com/github/asaad2k/FitersChatBot/blob/master/chatbotModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install Flask
!pip install flask-ngrok

In [ ]:
!pip install -U sentence-transformers
!pip install transformers

In [ ]:
!pip install pyngrok


In [ ]:
from flask import Flask, request, jsonify
import pandas as pd
import re
from sentence_transformers import SentenceTransformer, util
from pyngrok import ngrok

In [ ]:
# @title


# Initialize Flask app
app = Flask(__name__)

# Load Sentence Transformer model
sentence_model = SentenceTransformer('all-MiniLM-L6-v2')

# Load data from Excel
df = pd.read_excel('')
questions = df['Question'].tolist()
question_embeddings = sentence_model.encode(questions)

# Function to check if text contains a URL
def contains_url(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return bool(url_pattern.search(text))

# Function to calculate similarity
def get_similarity(user_query):
    query_embedding = sentence_model.encode(user_query)
    similarities = util.cos_sim(query_embedding, question_embeddings)

    if similarities.shape[1] == 0:
        return None, None

    most_similar_index = similarities.argmax().item()
    similarity_score = similarities[0][most_similar_index].item()

    return similarity_score, most_similar_index

# Function to retrieve the answer
def get_answer(most_similar_index):
    if 0 <= most_similar_index < len(df):
        answer = df['Answer'].iloc[most_similar_index]
        if contains_url(answer):
            return answer
        else:
            return answer
    else:
        return "No valid answer found."

# Route to handle POST requests for asking questions
@app.route('/ask', methods=['POST'])
def ask():
    user_input = request.json.get('question')
    similarity_score, most_similar_index = get_similarity(user_input)

    if most_similar_index is None:
        return jsonify({"error": "No valid similarity found."}), 400

    if 0 <= most_similar_index < len(df):
        if similarity_score >= 0.5:
            answer = get_answer(most_similar_index)
            return jsonify({"answer": answer, "similarity_score": similarity_score})
        else:
            return jsonify({"answer": "I don't have an answer for this question."})
    else:
        return jsonify({"error": "The computed index is out of bounds."}), 500

# Run Flask app on a different port (5001 to avoid conflicts)
if __name__ == '__main__':
    # Step 3: Expose the Flask Server with ngrok
    # Set up ngrok to tunnel the Flask server
    ngrok.set_auth_token('2kbLYnxcFcMehMwam0ebC2Ha1o8_5xgWyPRDXV5dX3HsaZ7Nr')  # Replace with your ngrok auth token
    public_url = ngrok.connect(5001)
    print(f"Public URL: {public_url}")

    # Run the Flask app
    app.run(port=5001)
